In [2]:
import exmel
from dataclasses import dataclass
from itertools import chain
from tqdm import tqdm
from exmel.score import MatchLike
from typing import TypedDict
import numpy as np
from collections import Counter
import pandas as pd

In [4]:
dataset = exmel.Dataset("dataset_v1")

In [5]:
class SongStat(TypedDict):
    name: str
    duration_per_event: float
    note_mean_song: float
    velocity_mean: float

song_stats: list[SongStat] = []

for song in dataset:
    melodies = song.melody.split()
    sum_duration, sum_events = 0, 0
    for m in melodies:
        sum_duration += m.duration
        sum_events += len(m)
    duration_per_event = sum_duration / sum_events
    note_mean = np.mean([event.note for event in song.performance.global_events])
    velocity_mean = np.mean([event.velocity for event in song.performance.global_events])
    song_stats.append(SongStat(
        name=song.name,
        duration_per_event=duration_per_event,
        note_mean_song=float(note_mean),
        velocity_mean=float(velocity_mean)))

In [6]:
song_df = pd.DataFrame(song_stats)
song_df.to_excel("song_stats.xlsx", index=False)

In [ ]:
def simple_len(m: MatchLike) -> int:
    return len(m.events)

In [4]:
alignments = exmel.inference_pipeline(
    exmel.AlignConfig(
        score_func=simple_len,
        hop_length=1,
        miss_tolerance=1,
        candidate_min_length=10,
        candidate_min_score=10),
    "dataset")

Processing songs: 100%|██████████| 17/17 [05:15<00:00, 18.57s/it]


In [5]:
sum_matches = sum(
    len(alignments[s].matches) + len(alignments[s].discarded_matches)
    for s in alignments)
print(sum_matches / len(alignments))

52558.529411764706


In [6]:
@dataclass(frozen=True, slots=True)
class Detection:
    tp: int
    fp: int
    start: float
    end: float

detections: dict[str, list[Detection]] = {}
for song in tqdm(dataset):
    detections[song.name] = []
    for match in chain(
        alignments[song.name].matches,
        alignments[song.name].discarded_matches,
    ):
        assert song.ground_truth is not None
        tp, fp = exmel.tp_fp(match, song.ground_truth)
        detections[song.name].append(
            Detection(tp, fp, match.start, match.end))

100%|██████████| 17/17 [06:47<00:00, 23.95s/it]


In [7]:
opts: dict[str, tuple[float, int]] = {}
for s, d in tqdm(detections.items()):
    gt = dataset[s].ground_truth
    assert gt is not None
    opt, subset = exmel.f1_optimal_interval_scheduling(
        d, num_gt=len(gt), epsilon=1e-6, return_subset=True)
    opts[s] = (opt, len(subset))

print(sum(opt for opt, _ in opts.values()) / len(opts))
print(sum(num for _, num in opts.values()) / len(opts))

100%|██████████| 17/17 [00:43<00:00,  2.57s/it]

0.9795447517843807
42.294117647058826


In [ ]:


class Entry(TypedDict):
    name: str
    tp: int
    fp: int
    length: int
    misses: int
    error: float
    velocity: float
    duration: float
    note_mean: float
    note_std: float
    note_entropy: float
    note_unique: int
    note_change: int

class ScoreFunctions:

    @staticmethod
    def length(match: MatchLike) -> int:
        return len(match.events)

    @staticmethod
    def misses(match: MatchLike) -> int:
        return match.sum_miss

    @staticmethod
    def error(match: MatchLike) -> float:
        return match.sum_error

    @staticmethod
    def velocity(match: MatchLike) -> float:
        return float(np.mean([event.velocity for event in match.events]))
    
    @staticmethod
    def duration(match: MatchLike) -> float:
        return match.events[-1].time - match.events[0].time

    @staticmethod
    def note_mean(match: MatchLike) -> float:
        return sum(event.note for event in match.events) / len(match.events)

    @staticmethod
    def note_std(match: MatchLike) -> float:
        return float(np.std([event.note for event in match.events]))

    @staticmethod
    def note_entropy(match: MatchLike) -> float:
        counts = np.array(list(Counter(event.note for event in match.events).values()))
        probs = counts / counts.sum()
        entropy = -np.sum(probs * np.log2(probs))
        return float(entropy)

    @staticmethod
    def note_unique(match: MatchLike) -> int:
        return len(set(event.note for event in match.events))

    @staticmethod
    def note_change(match: MatchLike) -> int:
        return sum(match.events[i].note != match.events[i - 1].note
                   for i in range(1, len(match.events)))

sf = ScoreFunctions

In [16]:
data: list[Entry] = []

for i, song in enumerate(dataset, 1):
    print(f"{i}/{len(dataset)}")
    for match in tqdm(chain(
        alignments[song.name].matches,
        alignments[song.name].discarded_matches,
    )):
        assert song.ground_truth is not None
        tp, fp = exmel.tp_fp(match, song.ground_truth)
        data.append(Entry(
            name=song.name,
            tp=tp,
            fp=fp,
            length=sf.length(match),
            misses=sf.misses(match),
            error=sf.error(match),
            velocity=sf.velocity(match),
            duration=sf.duration(match),
            note_mean=sf.note_mean(match),
            note_std=sf.note_std(match),
            note_entropy=sf.note_entropy(match),
            note_unique=sf.note_unique(match),
            note_change=sf.note_change(match)))

1/17


29407it [00:09, 3245.33it/s]


2/17


58140it [00:27, 2138.91it/s]


3/17


...

27366it [00:10, 2526.75it/s]


16/17


7143it [00:02, 2544.84it/s]


17/17


62924it [00:35, 1783.46it/s]


In [ ]:
df = pd.DataFrame(data)
df.to_csv("matches.csv", index=False, encoding="utf-8")